# Blue Brain BioExplorer

![](../bioexplorer_banner.png)

## Simplest script to create a visualization of a cell

In [ ]:
from bioexplorer import BioExplorer, Protein, Membrane, Cell, MolecularSystemAnimationParams, Vector2, Vector3, Quaternion
import os

be = BioExplorer('localhost:5000')
core = be.core_api()
status = be.reset_scene()
status = be.start_model_loading_transaction()

In [ ]:
import glob

name = 'Cell'
representation=be.REPRESENTATION_ATOMS_AND_STICKS

# Resources
resource_folder = os.path.abspath('../../tests/test_files')
pdb_folder = os.path.join(resource_folder, 'pdb')
membrane_folder = os.path.join(pdb_folder, 'membrane')
lipids_folder = os.path.join(membrane_folder, 'lipids')
transporters_folder = os.path.join(pdb_folder, 'transporters')

# ACE2 receptor definition
ace2_receptor = Protein(
    name=be.NAME_TRANS_MEMBRANE + '_ACE2',
    source=os.path.join(pdb_folder, '6m18.pdb'),
    occurrences=25,
    transmembrane_params=Vector2(-6.0, 6.0))

# GLUT3 definition
transporter = Protein(
    name=be.NAME_TRANS_MEMBRANE + '_GLUT3',
    source=os.path.join(transporters_folder, '4zwc.pdb'),
    rotation=Quaternion(0.707, 0.707, 0.0, 0.0),
    occurrences=50, chain_ids=[1],
    transmembrane_params=Vector2(-3.0, 3.0))

# Membrane definition
lipids = glob.glob(os.path.join(lipids_folder, '*.pdb'))[:4]

membrane = Membrane(
    load_non_polymer_chemicals=True, load_bonds=True,
    lipid_sources=lipids,
    lipid_density=1.0,
    animation_params=MolecularSystemAnimationParams(1, 1, 0.025, 2, 0.5)
)

# Cell definition
cell_size = Vector3(800.0, 50.0, 800.0)
cell = Cell(
    name=name,
    shape=be.ASSEMBLY_SHAPE_SINUSOID,
    shape_params=cell_size,
    membrane=membrane,
    proteins=[ace2_receptor, transporter])

# Add cell to scene
status = be.add_cell(
    cell=cell, atom_radius_multiplier=2.0, representation=representation)

In [ ]:
status = core.set_camera(
    orientation=[-0.070,  -0.041, 0.006, 0.996],
    position=[-76.69645577433823, 158.9458079662124, 970.5532710962468],
    target=[-14.769460404641611, 56.157789477305535, 243.45558273348504]
)

## Rendering settings

In [ ]:
status = be.core_api().set_renderer(
    background_color=[96 / 255, 125 / 255, 139 / 255],
    current='advanced',
    samples_per_pixel=1, subsampling=4, max_accum_frames=1000)
params = status = be.core_api().AdvancedRendererParams()
params.gi_samples = 1
params.gi_strength = 0.3
params.gi_ray_length = 5
params.shadow_intensity = 0.8
params.soft_shadow_strength = 1
params.fog_start = 1500
params.fog_thickness = 1500
params.max_ray_depth = 1
status = be.core_api().set_renderer_params(params)

## Materials

In [ ]:
be.apply_default_color_scheme(be.SHADING_MODE_BASIC)

In [ ]:
status = be.set_protein_color_scheme(
    assembly_name=name, name=be.NAME_TRANS_MEMBRANE + '_ACE2',
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Greens', palette_size=5)

status = be.set_protein_color_scheme(
    assembly_name=name, name=be.NAME_TRANS_MEMBRANE + '_GLUT3',
    color_scheme=be.COLOR_SCHEME_CHAINS,
    palette_name='Blues', palette_size=5)

In [ ]:
be.commit_model_loading_transaction()

## Modify receptor protein instance

In [ ]:
'''Get receptor protein instance transformation'''
instance_index = 0
transformation = be.get_protein_instance_transformation(
    assembly_name=name,
    name=be.NAME_TRANS_MEMBRANE + '_ACE2',
    instance_index=instance_index
)
print('Position = ' + str(transformation['position']))
print('Rotation = ' + str(transformation['rotation']))

In [ ]:
'''Modify receptor protein instance transformation'''
status = be.set_protein_instance_transformation(
    assembly_name=name,
    name=be.NAME_TRANS_MEMBRANE + '_ACE2',
    instance_index=instance_index,
    position=Vector3(-93.6, 20.0, 308.8),
    rotation=Quaternion(0.0182919, 0.0213303, 0.758803, 0.650714)
)

## Snapshot

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)
mm.create_snapshot(
    renderer='advanced', size=[1920, 1080], samples_per_pixel=64, 
    path='/tmp', base_name='cell')